In [36]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from google.cloud import bigquery
from datetime import datetime, timedelta

In [37]:
# Print current time
now = datetime.now()
current_time = now.strftime("%Y-%m-%d %H:%M:%S")

In [38]:
# Imports data
data = pd.read_csv('../processed_data/1988_to_2024_combined_clean_polling_and_results.csv')

In [39]:
data.rename(columns={'OTH_PERCENTAGE': 'OTH_ACT'}, inplace=True)

In [40]:
# Convert date columns to datetime
data['enddate'] = pd.to_datetime(data['enddate'])
data['next_elec_date'] = pd.to_datetime(data['next_elec_date'])
data['startdate'] = pd.to_datetime(data['startdate'])

In [41]:
# Handle numerical transformer
num_columns_selector = ['samplesize', 'months_to_elec_weight']
num_transformer = MinMaxScaler()

In [42]:
# Handle categorical transformer
cat_columns_selector = ['rating']
cat_transformer = make_pipeline(OrdinalEncoder(categories = [['F','F+','E-','E','E+','D-','D','D+','C-','C','C+','B-','B','B+','A-']]),MinMaxScaler())

In [43]:
# One-hot encoder for party_in_power
ohe = OneHotEncoder()

In [44]:
# Build the preprocessing pipeline
preproc_pipeline = make_column_transformer(
    (num_transformer, num_columns_selector),
    (cat_transformer, cat_columns_selector),
    (ohe,['party_in_power']),
    remainder='passthrough',
    verbose_feature_names_out=False
)

In [45]:
data['next_elec_date'].value_counts()

next_elec_date
2015-05-07    1930
2024-07-04    1362
2010-05-06     584
1992-04-09     453
2019-12-12     440
1997-05-01     338
2017-06-08     254
2001-06-07     213
2005-05-05      98
Name: count, dtype: int64

In [46]:
# Define election dates
election_date_2019 = datetime.strptime('2015-05-07', '%Y-%m-%d')
cutoff_date = election_date_2019 - timedelta(days=84)
prediction_date = election_date_2019 - timedelta(days=24)

In [47]:
# Split data
data_train = data[data['startdate'] < cutoff_date]
data_test_1 = data[(data['startdate'] >= cutoff_date) & (data['startdate'] < prediction_date)]
data_test = data_test_1[data_test_1['next_elec_date'] == election_date_2019]


In [48]:
# Fit transform preprocessing pipeline to data_train
data_train_processed = preproc_pipeline.fit_transform(data_train)


In [49]:
# Transform preprocessing pipeline to data_test
data_test_processed = preproc_pipeline.transform(data_test)

In [50]:
# Check feature names make sense for data_train
data_train_processed = pd.DataFrame(
    data_train_processed, columns=preproc_pipeline.get_feature_names_out()
)

# Check feature names make sense for data_test
data_test_processed = pd.DataFrame(
    data_test_processed, columns=preproc_pipeline.get_feature_names_out()
)

In [51]:
# Define our X by dropping irrelevant and y columns
X_train = data_train_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])
X_test = data_test_processed.drop(columns=['startdate', 'enddate', 'pollster', 'Unnamed: 0', 'next_elec_date', 'days_to_elec', 'months_to_elec', 'party_in_power_Labour', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT'])

In [52]:
# Build our target matrix
y_train = data_train_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]
y_test = data_test_processed[['next_elec_date', 'LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']]

In [53]:
# Drops y_train actuals where the actual is the actual result we are trying to predict, replaces with NaNs
y_train.loc[y_train['next_elec_date'] == '2015-05-07',
         ['LAB_ACT', 'CON_ACT', 'LIB_ACT', 'GRE_ACT', 'BRX_ACT', 'SNP_ACT', 'UKI_ACT', 'PLC_ACT', 'OTH_ACT']] = np.nan

In [68]:
X_test.tail(10)

,samplesize,months_to_elec_weight,rating,party_in_power_Conservative,poll_length,CON_FC,LAB_FC,LIB_FC,BRX_FC,GRE_FC,OTH_FC,PLC_FC,SNP_FC,UKI_FC
90,0.073605,0.831387,1.0,1.0,1,0.35,0.34,0.08,NaN,0.04,0.06,NaN,NaN,0.12
91,0.065631,0.831387,1.0,1.0,1,0.33,0.35,0.08,NaN,0.05,0.05,NaN,NaN,0.13
92,0.070998,0.831387,1.0,1.0,1,0.34,0.34,0.07,NaN,0.06,0.06,NaN,NaN,0.13
93,0.027806,0.831387,0.5,1.0,2,0.39,0.33,0.08,NaN,0.07,0.05,NaN,NaN,0.07
94,0.025813,0.831387,0.357143,1.0,2,0.33,0.33,0.09,NaN,0.06,0.08,NaN,NaN,0.13
95,0.078614,0.831387,0.5,1.0,2,0.33,0.33,0.08,NaN,0.05,0.06,NaN,NaN,0.15
96,0.062308,0.831387,1.0,1.0,1,0.33,0.36,0.07,NaN,0.05,0.06,NaN,NaN,0.13
97,0.035473,0.831387,0.428571,1.0,4,0.34,0.32,0.09,NaN,0.05,0.06,NaN,NaN,0.14
98,0.099468,0.831387,1.0,1.0,1,0.33,0.34,0.08,NaN,0.06,0.05,NaN,NaN,0.13
99,0.025659,0.831387,1.0,1.0,3,0.33,0.35,0.07,NaN,0.08,0.07,NaN,NaN,0.1


In [54]:
# Calculates average median value of X_test
averages = X_test.mean()
averages

samplesize                     0.059869
months_to_elec_weight          0.742303
rating                         0.816429
party_in_power_Conservative         1.0
poll_length                        1.59
CON_FC                           0.3318
LAB_FC                           0.3359
LIB_FC                           0.0784
BRX_FC                              NaN
GRE_FC                            0.057
OTH_FC                            0.056
PLC_FC                              NaN
SNP_FC                              NaN
UKI_FC                           0.1409
dtype: object

In [55]:
# Creates imputation values for y_train to impute over actuals we are trying to predict
imputation_values = {
    'CON_ACT': averages['CON_FC'],
    'LAB_ACT': averages['LAB_FC'],
    'LIB_ACT': averages['LIB_FC'],
    'BRX_ACT': 0, # doesnt exist in 2015
    'GRE_ACT': averages['GRE_FC'],
    'OTH_ACT': averages['OTH_FC'],
    'PLC_ACT':    0.006,
    'SNP_ACT': 0.04, # imputes value due to low forecasts
    'UKI_ACT': averages['UKI_FC']
}

In [56]:
# Applies imputation values to y_train
y_train = y_train.fillna(value=imputation_values)


/tmp/ipykernel_138859/3014033841.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.fillna(value=imputation_values)


In [57]:
# Instantiate the model for each party
models = {}
parties = ['CON', 'LAB', 'LIB', 'BRX', 'GRE', 'SNP', 'UKI', 'PLC', 'OTH']
for party in parties:
    models[party] = XGBRegressor(
        learning_rate=0.3, n_estimators=300, max_depth=3, subsample=0.7,
        objective='reg:squarederror', nthread=-1, enable_categorical=True
    )

In [58]:
# Train models
X_train_matrix = np.array(X_train)
for party in parties:
    models[party].fit(X_train_matrix, y_train[f'{party}_ACT'])

In [59]:
# Evaluate model performance with cross validation
cv_scores = {}
for party in parties:
    cv_scores[party] = cross_val_score(models[party], X_train_matrix, y_train[f'{party}_ACT']).mean()

In [60]:
actuals_2015 = {
    "CON": 0.368,  # Conservative Party
    "LAB": 0.307,  # Labour Party
    "LIB": 0.079,  # Liberal Democrats
    "BRX": 0.000,  # Brexit Party (did not exist in 2015)
    "GRE": 0.038,  # Green Party
    "SNP": 0.050,  # Scottish National Party
    "UKI": 0.126,  # UK Independence Party
    "PLC": 0.005,  # Plaid Cymru
    "OTH": 0.027   # Other parties
}

In [61]:
# Evaluate predictions
X_test_matrix = np.array(X_test)
mean_predictions = {}
for party in parties:
    mean_predictions[party] = models[party].predict(X_test_matrix).mean()


In [62]:
# Print mean predictions
for party in parties:
    print(f"2015 mean prediction for {party}: {mean_predictions[party]}")
    print(f"2015 actual result for {party}  : {actuals_2015[party]}")


2015 mean prediction for CON: 0.3226492404937744
2015 actual result for CON  : 0.368
2015 mean prediction for LAB: 0.33812469244003296
2015 actual result for LAB  : 0.307
2015 mean prediction for LIB: 0.0791584700345993
2015 actual result for LIB  : 0.079
2015 mean prediction for BRX: 0.0
2015 actual result for BRX  : 0.0
2015 mean prediction for GRE: 0.05669592320919037
2015 actual result for GRE  : 0.038
2015 mean prediction for SNP: 0.039999037981033325
2015 actual result for SNP  : 0.05
2015 mean prediction for UKI: 0.14074283838272095
2015 actual result for UKI  : 0.126
2015 mean prediction for PLC: 0.005976035725325346
2015 actual result for PLC  : 0.005
2015 mean prediction for OTH: 0.06096232309937477
2015 actual result for OTH  : 0.027


In [63]:
# Evaluate last five polls
last_predictions = {}
for party in parties:
    last_predictions[party] = models[party].predict(X_test_matrix)[-5:]

In [66]:
# Print mean of predictions
for party in parties:
    print(f"2015 last prediction for {party}: {last_predictions[party].mean()}")
    print(f"2015 actual result for {party}  : {actuals_2015[party]}")


2015 last prediction for CON: 0.3053290843963623
2015 actual result for CON  : 0.368
2015 last prediction for LAB: 0.3406062722206116
2015 actual result for LAB  : 0.307
2015 last prediction for LIB: 0.07993648946285248
2015 actual result for LIB  : 0.079
2015 last prediction for BRX: 0.0
2015 actual result for BRX  : 0.0
2015 last prediction for GRE: 0.05651453137397766
2015 actual result for GRE  : 0.038
2015 last prediction for SNP: 0.04000129550695419
2015 actual result for SNP  : 0.05
2015 last prediction for UKI: 0.14088746905326843
2015 actual result for UKI  : 0.126
2015 last prediction for PLC: 0.005984189920127392
2015 actual result for PLC  : 0.005
2015 last prediction for OTH: 0.06971053779125214
2015 actual result for OTH  : 0.027
